## 导入模型

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# 原始模型路径和训练后的LoRA路径
base_model_path = "/root/autodl-tmp/Qwen/Qwen3-8B"  # 原始模型路径
lora_model_path = "./lora_model"                    # 你保存的LoRA模型路径

# ---------- 1. 加载分词器 ----------
tokenizer = AutoTokenizer.from_pretrained(lora_model_path)  # 直接加载你保存的分词器
# tokenizer = AutoTokenizer.from_pretrained(base_model_path)  # 直接加载你保存的分词器

# ---------- 2. 加载基础模型 ----------
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto"  # 自动分配设备（GPU/CPU） 这里只是放在一个GPU上面
)

# ---------- 3. 加载LoRA适配器 ----------
model = PeftModel.from_pretrained(base_model, lora_model_path)
model.eval()  # 切换到推理模式

## 保存模型的方式

In [ ]:
# 保存为float16格式（用于VLLM）
# 支持多种保存方式：merged_16bit（float16）、merged_4bit（int4）或lora（适配器）
save_path = "./saved_models/qwen3_4bit"  # 本地保存路径
# Merge to 16bit
if False:
    model.save_pretrained_merged(save_path, tokenizer, save_method = "merged_16bit",)
if False: # 上传到HuggingFace Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 保存为4位精度
if True:
    model.save_pretrained_merged(save_path, tokenizer, save_method = "merged_4bit_forced",) # 改为_forced版本
if True: # 上传到HuggingFace Hub
    model.push_to_hub_merged("yukisama123/Qwen3-vLLM", tokenizer, save_method = "merged_4bit_forced", token = "") # 同样改为_forced版本

# 仅保存LoRA适配器
if False:
    model.save_pretrained_merged(save_path, tokenizer, save_method = "lora",)
if False: # 上传到HuggingFace Hub
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [ ]:
# GGUF / llama.cpp 格式转换
# 支持多种量化方法，如q8_0、q4_k_m、q5_k_m等

# F16（Float16）格式

# 精度类型：半精度浮点数（16位浮点数）
# 内存占用：比原始FP32（32位浮点数）减少约50%的存储空间
# 精度保留：保留了相对较高的数值精度，损失较小
# 推理性能：比FP32快，但比更低位量化格式慢
# 适用场景：当需要在内存使用和模型精度之间取得平衡时使用

# Q4_K_M格式

# 精度类型：混合4位量化格式（是GGUF量化方案的一种）
# 内存占用：比F16减少约75%的存储空间，比原始FP32减少约87.5%
# 量化策略：针对不同权重采用不同的量化策略

# 对注意力机制中的WV矩阵和前馈网络中的W2矩阵的一半使用Q6_K量化
# 对其余权重使用Q4_K量化

# 精度与速度：牺牲一定精度以获得更小的文件大小和更快的推理速度
# 适用场景：适合在资源受限设备上运行模型，如个人电脑或移动设备

# # Save to 8bit Q8_0
# if False:
#     model.save_pretrained_gguf("model", tokenizer,)
# # Remember to go to https://huggingface.co/settings/tokens for a token!
# # And change hf to your username!
# if False:
#     model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# # 保存为16位GGUF
# if False:
#     model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
# if False: # 上传到HuggingFace Hub
#     model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# # 保存为q4_k_m格式GGUF
if True:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if True:# 上传到HuggingFace Hub
    model.push_to_hub_gguf("yukisama123/Qwen3-GGUF", tokenizer, quantization_method = "q4_k_m", token = "")

# # 保存多种GGUF格式（批量导出更高效）
# if False:
#     model.push_to_hub_gguf(
#         "hf/model", # Change hf to your username!
#         tokenizer,
#         quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
#         token = "", # Get a token at https://huggingface.co/settings/tokens
#     )